<a href="https://colab.research.google.com/github/mosdef-hub/CECAM-MoSDeF-Workshop/blob/main/biomolecule_workflow/Biomolecule-Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoSDeF Biomolecule Workflow

## Tutorial summary
This tutorial aims to familiarize a molecular simulations researcher on the ways to create a complex molecule recipe and build-up an initial system using that recipe. In addition, after creating the inital system we demonstrate how to identify missing forcefield parameters and parameterize a system using [Foyer](https://github.com/mosdef-hub/foyer) and [GMSO](https://github.com/mosdef-hub/gmso).  This demonstrates the strengths of using the [MoSDeF](https://github.com/mosdef-hub) simulation framework for creating modular, complex, and custom molecular systems for computational chemistry simulations.

## Learning Objectives:
* How to subclass an `mBuild.Compound` to create your own recipe
* How to use visualization to identify specific atoms in a [mBuild.Compound](https://github.com/mosdef-hub/mbuild/blob/468028b5d0185c7325f91ee4fce7e50e73d1306d/mbuild/compound.py#L57)
* How to use gmso to debug parameterization
* How to add missing parameters to forcefield xmls

## Exercise Stages:
0. Setup enviroment on Google Colab and import libraries
1. Load molecules and create molecules
    1. Exercise 1: Creating a simple recipe by identifying specific atoms by index
    2. Exercise 2: Creating a more complex recipe by idenfiying atoms by method name
    3. Exercise 3: Use our complex recipe to build a Chimeric spider silk protein
2. Pack a box of solvated protein
3. Load forcefield
4. Apply forcefield
    1. Exercise 4: Forcefield correction
5. Reload forcefield
6. Write `Hoomd`
7. Run `Hoomd`
8. Analyize trajectory results
---

## 0. Set up environment on Google Colab
---

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniforge()

Run the cell below again after restart.

In [ ]:
import condacolab
condacolab.check()

!conda install mamba
!mamba install anaconda-client -n base

!git clone https://github.com/mosdef-hub/CECAM-MoSDeF-Workshop

!git clone https://github.com/kierannp/protein_builder.git
%cd protein_builder
!pip install .
%cd ..
!pip install --upgrade ipykernel

!mamba install -c conda-forge py3Dmol nglview mbuild hoomd unyt forcefield-utilities fresnel gsd openbabel
!mamba install gmso=0.11.2

In [ ]:
# Import Libraries
import mbuild as mb
import gmso
import hoomd
from pBuilder import Arginine, Glycine, Glutamine, Leucine, Alanine # R, G, Q, L, A

import io
import warnings

import fresnel
import IPython
import numpy
import packaging.version
import PIL
import gsd.hoomd
import math
import numpy as np

## 1. Load Molecules
---

We can visualize the monomer units we will use to make our protein.

In [ ]:
mol = Glycine() # You can replace this line with different amino acids listed above to see what they look like
mol.visualize()

Sometimes NGLView is preferable as a visualization engine, since it highlights index number by hovering over the atoms.

In [ ]:
#these line enable the nglviewer in colab
from google.colab import output
output.enable_custom_widget_manager()

mol.visualize(backend='NGLView')

## 1.1 Protein Background
---

Amino acids have both amino and carboxylic acid functional groups and possess a generic structure as seen below:

![image info](https://github.com/mosdef-hub/CECAM-MoSDeF-Workshop/blob/main/images/Amino_acid_generic_structure.png?raw=1)

Proteins are polypeptide polymers that are held together by peptide bonds occuring between amino acids. Proteins have ends specificed as N-terminus and C-terminus, the green and blue molecular regions in the image below, respectively.

<img src="https://github.com/mosdef-hub/CECAM-MoSDeF-Workshop/blob/main/images/Tetrapeptide_structural_formulae_v.1.png?raw=1" alt="C/N-terminus" width="600"/>

## 1.2 Subclassing `mBuild.Compound` to create modular molecules

If we wish to develop a program that can make a protein from any sequence of amino acids we can utilize `mBuild`!

First we can start with building a single amino acid from a smiles string. Not the just_name data object is just a dictionary that holds all amino acid SMILES strings given the name of the amino acid as the key.

In [ ]:
from pBuilder.all_names import *

just_name['alanine'] #this data structure contains the SMILES names of all canonical amino acids

Here is an example of building an mBuild.Compound from a SMILES string and associating certain atoms in that molecule with varibles.

In [ ]:
from pBuilder.all_names import *

# Here is an example of the various amino acids that are used to create the protein
class Alanine(mb.Compound):
    def __init__(self):
        super(Alanine, self).__init__()
        alanine = mb.load(just_name['alanine'],smiles=True)
        self.add(alanine)
        self.name = "Alanine"
        self.amine = alanine[5]
        self.carboxyl = alanine[2]
        amine_h = [alanine[11], alanine[12]]
        carboxyl_o = alanine[3]
        self.indices = [amine_h, carboxyl_o]

# Exercise 1: Creating a simple recipe by identifying specific atoms by index
Correct the code below by specifying the correct atom indices <br><br>
*Hint: Visualize Glycine using `NGLView` to see which atoms indices need to specifed to mark part of the amino acid*

In [ ]:
class Glycine(mb.Compound):
    def __init__(self):
        super(Glycine, self).__init__()
        glycine = mb.load(just_name['glycine'],smiles=True)
        self.add(glycine)
        self.name = "Glycine"

        # TODO identify the correct molecules in the glycine mBuild.Compound
        self.amine = ???
        self.carboxyl = ???
        amine_h = ???
        carboxyl_o = ???
        self.indices = [amine_h, carboxyl_o]

In [ ]:
glycine = mb.load(just_name['glycine'],smiles=True)
glycine.visualize(backend='NGLView')

### <font color="red"><b>Exercise 1 Answer</b></font>

<details>
    <summary>Click once to show answer!</summary>

        class Glycine(mb.Compound):
        def __init__(self):
            super(Glycine, self).__init__()
            glycine = mb.load(just_name['glycine'],smiles=True)
            self.add(glycine)
            self.name = "Glycine"
            self.amine = glycine[4]
            self.carboxyl = glycine[1]
            amine_h = [glycine[8], glycine[9]]
            carboxyl_o = glycine[3]
            self.indices = [amine_h, carboxyl_o]

</details>

# Exercise 2: Creating a more complex recipe by idenfiying atoms by method name

Once we have all the amino acids `mBuild.Compound` molecules created (as is implemented in the `pBuilder` package) we can create a `mBuild` class for building any generic protein. <br> <br>See if you can implement the remove_N_terminal method below.

In [ ]:
from pBuilder.utils import *
from pBuilder.aminos import *

class Protein(mb.Compound):
    def __init__(self):
        super().__init__()

    def build(self, seq):
        """
        Builds up a protein from a sequence of amino acids
        """
        start_mol = symbol_class[seq[0]]()
        N_terminal = self.remove_C_terminal(start_mol)
        previous_acid = N_terminal
        self.add(previous_acid,label='N-terminal')

        for i, letter in enumerate(seq[1:-1]):
            current_acid = symbol_class[letter]()
            current_acid = self.remove_both_terminals(current_acid)
            N_port = mb.Port(anchor=current_acid.amine,
   orientation=[0, 1, -1],
   separation=0.1)
            C_port = mb.Port(anchor=current_acid.carboxyl,
   orientation=[1, 0, 0],
   separation=0.1)
            current_acid.add(N_port, label='head')
            current_acid.add(C_port, label='tail')
            # current_acid['head'].rotate(around=[-10,0,1], theta=(i+1)*np.pi/4)
            mb.force_overlap(move_this=current_acid,
   from_positions=current_acid['head'],
   to_positions=previous_acid['tail'])
            self.add(current_acid, label='aa_'+str(i+1))
            previous_acid = current_acid

        final_mol = symbol_class[seq[-1]]()
        C_terminal = self.remove_N_terminal(final_mol)

        mb.force_overlap(move_this=C_terminal,
            from_positions=C_terminal['head'],
            to_positions=previous_acid['tail'])
        self.add(C_terminal,label='C-terminal')

    def remove_C_terminal(self, mol):
        """
        Removes the C_terminal from an amino acid mBuild.Compound and returns the clipped amino acid
        """
        amine_h, carboxyl_o = mol.indices
        mol.remove(carboxyl_o)
        mol.add(mb.Port(anchor=mol.carboxyl,
     orientation=[0, 1, 0],
     separation=0.1),
          label='tail')
        mol.spin(around=[0,1,0], theta=np.pi/2)
        return mol

    def remove_N_terminal(self, mol):
        """
        Removes the N_terminal from an amino acid mBuild.Compound and returns the clipped amino acid
        """
        # TODO get this method working!

    def remove_both_terminals(self, mol):
        """
        Removes both C and N terminus from the amino acid and returns clipped amino acid, this is used for amino acids in the middle of the protein
        """
        amine_h, carboxyl_o = mol.indices
        mol.remove(amine_h[0])
        mol.remove(carboxyl_o)
        return mol


### <font color="red"><b>Exercise 2 Answer</b></font>


<details>
    <summary>Click once show the answer!</summary>

    def remove_N_terminal(self, mol):
        """
        Removes the N_terminal from an amino acid mBuild.Compound and returns the clipped amino acid
        """
        amine_h, carboxyl_o = mol.indices
        mol.remove(amine_h[0])
        mol.add(mb.Port(anchor=mol.amine,
     orientation=[-1, .7, -.2],
     separation=0.05),
          label='head')
        return mol

</details>

# Exercise 3: Use our complex recipe to build a Chimeric spider silk protein

Here we are looking at a chimeric spider silk protein. This protein consists of a nonrepeat sequence of amino acids followed by a varible number of a repeated amino acid sequence. See if you can create something resembling the [NT2RepCT](https://www.nature.com/articles/nchembio.2269) spider silk protein. <br><br>
*Hint: Utilize the `build` method in the Protein recipe and string manipulation*

In [ ]:
# Uncomment the line below if you were unable to get the Protein class functioning
# from pBuilder.polypeptide import Protein

In [ ]:
n_repeats = 2
chain = Protein()
nonrepeat = 'GGQGGAGQGGYGGLGSQGAGRGGLGGQ'
repeat = 'GAGAAAAAAGGAGQGGTGGLGSQGAGRGGL'
chain.name="Protein"
# TODO create the sequence we want
chain.translate(-chain.center) #translate_to, rotate, spin, rotate_dihedral
chain.visualize()

### <font color="red"><b>Exercise 3 Answer</b></font>


<details>
    <summary>Click once on to hide/unhide the answer!</summary>

    chain.build(nonrepeat+repeat*n_repeats + nonrepeat)

</details>

## 2. Combine and Solvate
---

Now that we have our protein build we can solvate it to try to get a native fold.

In [ ]:
rotable_bond = list(chain.bonds())[625]
chain.rotate_dihedral(bond=rotable_bond, phi=3.14) #translate_to, rotate, spin, rotate_dihedral
# control energy minimization
chain.energy_minimize(steps=50)
chain.visualize()

In [ ]:
water = mb.load("O", smiles=True)
water.name="H2O"
packed_box = mb.fill_box([chain, water], n_compounds=[1,1000], box=[10,10,10])
print(packed_box.print_hierarchy(show_tree=False))
packed_box.visualize()

In [ ]:
# save out and reload current state for future use
packed_box.save("solvated_protein.pdb", overwrite=True)
reloaded_pdb = mb.load("solvated_protein.pdb") #xyz, gro, lammpsdata, sdf, mol2, hoomdxml, json

In [ ]:
! head "solvated_protein.pdb"

## 3. Load Two ForceFields
---

A `MoSDeF` compatible implementation of the Generalized Amber Forcefield (GAFF) can be found [here](https://github.com/rsdefever/GAFF-foyer).

In [ ]:
gaff_forcefield = gmso.ForceField("./CECAM-MoSDeF-Workshop/forcefields/gaff.xml")
gaff_forcefield

In [ ]:
spce_forcefield = gmso.ForceField("./CECAM-MoSDeF-Workshop/forcefields/spce.xml")
spce_forcefield

## 4. Apply Forcefield
---

In [ ]:
from gmso.parameterization import apply

gmso_top = packed_box.to_gmso()
forcefield_matchingDict = {"Protein":gaff_forcefield, "H2O":spce_forcefield}
gmso_top = packed_box.to_gmso()
parameterized_top = apply(
    gmso_top, forcefield_matchingDict, identify_connections=True,
)

### What Happened
This error indicates that have particles in our mbuild system that are missing parameters in our xml forcefield file. We will show how to correct this below.

# Exercise 4: Forcefield correction

See if you can figure out which lines to add to the forcefield file (gaff.xml) to get this to properly parameterize our system. <br><br>
*Hint: We suggest substituting the parameters (c1,c3,n3) for the (c1, c3, n2) parameters and (c3,c2,oh) for the (c3, c1, oh) parameters*

In [ ]:
import unyt as u
from gmso.lib.potential_templates import PotentialTemplateLibrary  

ff = gmso.ForceField("./CECAM-MoSDeF-Workshop/forcefields/gaff.xml")        
templates = PotentialTemplateLibrary()  
harmonic_angle = templates["HarmonicAnglePotential"]
expression = harmonic_angle.expression  
name = harmonic_angle.name
variables = harmonic_angle.independent_variables     
first_params = {  
    "k": 562.3296 * u.kJ/u.mol/u.rad**2, #these parameters were substituted from (c1,c3,n3) from the link below
    "theta_eq":1.9675096657732076*u.rad
}
second_params = ???
        
first_angle_type = gmso.AngleType(        
    name=name, parameters=first_params,       
    expression=expression,
    independent_variables=variables,    
    member_classes = ("c1", "c3", "n2")
)
second_angle_type = ???

ff.angle_types["c1~c3~n2"] = first_angle_type    
ff.angle_types["c3~c1~oh"] = ??? 
ff.version = 1.1 # updated version since we made modifications to the forcefield
ff.name = "gaff with added parameters from https://github.com/choderalab/ambermini/blob/master/share/amber/dat/leap/parm/gaff.dat"
ff.to_xml("gaff_added.xml")

### <font color="red"><b>Exercise 4 Answer</b></font>


<details>
    <summary>Click once to show the answer!</summary>
      
      import unyt as u
      from gmso.lib.potential_templates import PotentialTemplateLibrary 
      ff = gmso.ForceField("./CECAM-MoSDeF-Workshop/forcefields/gaff.xml")        
      templates = PotentialTemplateLibrary()  
      harmonic_angle = templates["HarmonicAnglePotential"]
      expression = harmonic_angle.expression  
      name = harmonic_angle.name
      variables = harmonic_angle.independent_variables     
      first_params = {  
          "k": 562.3296 * u.kJ/u.mol/u.rad**2,
          "theta_eq":1.9675096657732076*u.rad
      }
      second_params = {  
          "k": 571.5344 * u.kJ/u.mol/u.rad**2,
          "theta_eq":2.0099211665966696*u.rad
      }
                
      first_angle_type = gmso.AngleType(        
          name=name, parameters=first_params,       
          expression=expression,
          independent_variables=variables,    
          member_classes = ("c1", "c3", "n2")
      )
      second_angle_type = gmso.AngleType(        
          name=name, parameters=second_params,       
          expression=expression,
          independent_variables=variables,    
          member_classes = ("c3", "c1", "oh")
      )

      ff.angle_types["c1~c3~n2"] = first_angle_type
      ff.angle_types["c3~c1~oh"] = second_angle_type          
      ff.version = 1.1                                                                                                                                          ff.name = "gaff with added parameters from https://github.com/choderalab/ambermini/blob/master/share/amber/dat/leap/parm/gaff.dat"
      ff.to_xml("gaff_added.xml")

</details>

## 5. Reload and apply the forcefield
---

We are assuming that you saved the new forcefield xml as gaff2.xml. If you weren't able to get the forcefield working please use gaff_ANSWER.xml.

In [ ]:
gaff_forcefield = gmso.ForceField("./CECAM-MoSDeF-Workshop/forcefields/gaff_ANSWER.xml")
gmso_top = packed_box.to_gmso()
forcefield_matchingDict = {"Protein":gaff_forcefield, "H2O":spce_forcefield}
parameterized_top = apply(
    gmso_top, forcefield_matchingDict, identify_connections=True,
)
gmso_top._impropers = [] # we are removing the impropers for this simulation since hoomd doesn't support Improper PeriodicTorsionPotential yet

## 6. Write HOOMD Objects
---

In [ ]:
#parameterized_top.save("top.gsd", overwrite=True)
import unyt as u

from gmso.external import to_hoomd_forcefield, to_hoomd_snapshot

base_units = {
    "mass": u.g / u.mol,
    "length": u.nm,
    "energy": u.kJ / u.mol,
}

gmso_snapshot, snapshot_base_units = to_hoomd_snapshot(
    parameterized_top, base_units=base_units
)
gmso_forces, forces_base_units = to_hoomd_forcefield( #can't handle dimensionless parameters currently, PR incoming
    parameterized_top,
    r_cut=1.4,
    base_units=base_units,
    pppm_kwargs={"resolution": (64, 64, 64), "order": 7},
)
gmso_forces

## 7. Run HOOMD Simulations
---

In [ ]:
temp = 300 * u.K
kT = temp.to_equivalent("kJ/mol", "thermal").value

cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)
# sim.create_state_from_gsd("top.gsd") # does not work
sim.create_state_from_snapshot(gmso_snapshot)
sim.operations.integrator = hoomd.md.Integrator(dt=0.0001)
sim.operations.integrator.forces.extend(
    list(set().union(*gmso_forces.values()))[:-1]
)
bussi = hoomd.md.methods.thermostats.Bussi(kT=kT)
nvt = hoomd.md.methods.ConstantVolume(
    filter=hoomd.filter.All(),
    thermostat=bussi
)
sim.operations.integrator.methods.append(nvt)

sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=kT)
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
    filter=hoomd.filter.All()
)

sim.operations.computes.append(thermodynamic_properties)
logger = hoomd.logging.Logger()
logger.add(thermodynamic_properties)
import os
if os.path.exists('trajectory.gsd'):
    os.remove("trajectory.gsd")
gsd_writer = hoomd.write.GSD(
    filename='trajectory.gsd',
    trigger=hoomd.trigger.Periodic(50),
     mode='xb',
     filter=hoomd.filter.All(),
     logger=logger
)
sim.operations.writers.append(gsd_writer)
outlogger = hoomd.logging.Logger(categories=['scalar', 'string'])
outlogger.add(sim, quantities=['timestep', 'tps'])
outlogger.add(thermodynamic_properties, ['kinetic_temperature'])
table = hoomd.write.Table(
    trigger=hoomd.trigger.Periodic(period=50),
    logger=outlogger
)
sim.operations.writers.append(table)
sim.run(1000)

## 8. Analyze results
---

In [ ]:
device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=300, h=300)

FRESNEL_MIN_VERSION = packaging.version.parse("0.13.0")
FRESNEL_MAX_VERSION = packaging.version.parse("0.14.0")


protein = np.array(list(range(954)))

def render(snapshot, particles=None, is_solid=None, indices=None):
    if ('version' not in dir(fresnel) or packaging.version.parse(
            fresnel.version.version) < FRESNEL_MIN_VERSION
            or packaging.version.parse(
   fresnel.version.version) >= FRESNEL_MAX_VERSION):
        warnings.warn(
            f"Unsupported fresnel version {fresnel.version.version} - expect errors."
        )
    vertices = [
        (-0.5, 0, 0),
        (0.5, 0, 0),
        (0, -0.5, 0),
        (0, 0.5, 0),
        (0, 0, -0.5),
        (0, 0, 0.5),
    ]
    # poly_info = fresnel.util.convex_polyhedron_from_vertices(vertices)
    N = snapshot.particles.N if indices is None else len(indices)
    L = snapshot.configuration.box[0]
    if particles is not None:
        N = len(particles)
    if is_solid is not None:
        N = int(numpy.sum(is_solid))

    scene = fresnel.Scene(device)
    # geometry = fresnel.geometry.ConvexPolyhedron(scene, poly_info, N=N)
    geometry = fresnel.geometry.Sphere(
        scene,
        N=N,
        radius = .05
    )
    geometry.material = fresnel.material.Material(color=fresnel.color.linear(
        [0.01, 0.74, 0.26]),
           roughness=0.5)
    if indices is None:
        geometry.position[:] = snapshot.particles.position[:]
    else:
        geometry.position[:protein.shape[0]] = snapshot.particles.position[:protein.shape[0]]


    geometry.outline_width = 0.01
    box = fresnel.geometry.Box(scene,
     snapshot.configuration.box,
     box_radius=.02)

    scene.lights = [
        fresnel.light.Light(direction=(0, 0, 1),
  color=(0.8, 0.8, 0.8),
  theta=math.pi),
        fresnel.light.Light(direction=(1, 1, 1),
  color=(1.1, 1.1, 1.1),
  theta=math.pi / 3)
    ]
    scene.camera = fresnel.camera.Orthographic(position=(L * 2, L, L * 2),
        look_at=(0, 0, 0),
        up=(0, 1, 0),
        height=L * 1.4 + 1)
    scene.background_color = (1, 1, 1)
    return tracer.sample(scene, samples=10)


def render_movie(frames, particles=None, is_solid=None, indices = None):
    if is_solid is None:
        is_solid = [None] * len(frames)
    a = render(frames[0], indices=indices)

    im0 = PIL.Image.fromarray(a[:, :, 0:3], mode='RGB').convert(
        "P", palette=PIL.Image.Palette.ADAPTIVE)
    ims = []
    for i, f in enumerate(frames[1:]):
        a = render(f, indices=indices)
        im = PIL.Image.fromarray(a[:, :, 0:3], mode='RGB')
        im_p = im.quantize(palette=im0)
        ims.append(im_p)

    blank = numpy.ones(shape=(im0.height, im0.width, 3), dtype=numpy.uint8) * 255
    im = PIL.Image.fromarray(blank, mode='RGB')
    im_p = im.quantize(palette=im0)
    ims.append(im_p)

    f = io.BytesIO()
    im0.save(f, 'gif', save_all=True, append_images=ims, duration=1000, loop=0)

    size = len(f.getbuffer()) / 1024
    if (size > 3000):
        warnings.warn(f"Large GIF: {size} KiB")
    return IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
traj = gsd.hoomd.open('./CECAM-MoSDeF-Workshop/biomolecule_workflow/pre_run_trajectory.gsd') # replace with trajectory.gsd to view your run
render_movie(traj, indices = protein)